In [ ]:
import ast


def str_node(node, info, level):
    info["level"] = level
    if isinstance(node, ast.AST):
        for name, val in ast.iter_fields(node):
                if "ast" in str(val):
                    info[name] = None
                else:
                    info[name] = val
        fields = [(name, str_node(val,info,level)) for name, val in ast.iter_fields(node) if name not in ('left', 'right')]
        info["type"] = node.__class__.__name__
        #print(info)
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        #print(repr(node))
        return repr(node)

def ast_visit(node, state_types,level=0):
    #print('  ' + str(level)  + str_node(node,{},level) )
    info = {}
    str_node(node,info, level )
    print(info)
    
    for field, value in ast.iter_fields(node):
        #print(field)
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item,state_types, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value,state_types, level=level+1)


state_types = []
with open("data/ast_example.py") as f:
    code = f.read()
    ast_visit(ast.parse(code),state_types)